In [1]:
import os
import time
import json
import requests
import platform
import browser_cookie3
from playsound import playsound

In [2]:
def load_cookie_from_chrome():
    global jar

    cookie_path = None
    os_type = platform.system()
    if os_type == "Linux":
        # browser_cookie3 also checks beta version of google chrome's cookie file.
        cookie_path = os.path.expanduser(
            "~/.config/google-chrome/Default/Cookies")
        if os.path.exists(cookie_path) is False:
            cookie_path = os.path.expanduser(
                "~/.config/google-chrome-beta/Default/Cookies")
    elif os_type == "Darwin":
        cookie_path = os.path.expanduser(
            "~/Library/Application Support/Google/Chrome/Default/Cookies")
    elif os_type == "Windows":
        cookie_path = os.path.expandvars(
            "%LOCALAPPDATA%/Google/Chrome/User Data/Default/Cookies")
    else:  # Jython?
        print("지원하지 않는 환경입니다.")
        close()

    if os.path.exists(cookie_path) is False:
        print("기본 쿠키 파일 경로에 파일이 존재하지 않습니다. 아래 링크를 참조하여 쿠키 파일 경로를 지정해주세요.\n" +
              "https://github.com/SJang1/korea-covid-19-remaining-vaccine-macro/discussions/403")
        close()

    cookies = browser_cookie3.chrome(domain_name=".naver.com")
    return cookies

In [3]:
def find_vaccine(cookie):
    global done
    
    url = "https://api.place.naver.com/graphql"
    header = {'Content-Type': 'application/json; charset=utf-8'}
    payload = '[{"operationName":"vaccineList","variables":{"input":{"keyword":"코로나백신위탁의료기관","categories":["1004836"],"x":"129.3487515","y":"36.0227425"},"businessesInput":{"start":0,"display":100,"deviceType":"mobile","x":"129.3487515","y":"36.0227425","bounds":"129.3289675;36.0005255;129.3685355;36.0449533","sortingOrder":"distance"},"isNmap":false,"isBounds":false},"query":"query vaccineList($input: RestsInput, $businessesInput: RestsBusinessesInput, $isNmap: Boolean!, $isBounds: Boolean!) {  rests(input: $input) {    businesses(input: $businessesInput) {      total      vaccineLastSave      isUpdateDelayed      items {        id        name        dbType        phone        virtualPhone        hasBooking        hasNPay        bookingReviewCount        description        distance        commonAddress        roadAddress        address        imageUrl        imageCount        tags        distance        promotionTitle        category        routeUrl        businessHours        x        y        imageMarker @include(if: $isNmap) {          marker          markerSelected          __typename        }        markerLabel @include(if: $isNmap) {          text          style          __typename        }        isDelivery        isTakeOut        isPreOrder        isTableOrder        naverBookingCategory        bookingDisplayName        bookingBusinessId        bookingVisitId        bookingPickupId        vaccineOpeningHour {          isDayOff          standardTime          __typename        }        vaccineQuantity {          totalQuantity          totalQuantityStatus          startTime          endTime          vaccineOrganizationCode          list {            quantity            quantityStatus            vaccineType            __typename          }          __typename        }        __typename      }      optionsForMap @include(if: $isBounds) {        maxZoom        minZoom        includeMyLocation        maxIncludePoiCount        center        __typename      }      __typename    }    queryResult {      keyword      vaccineFilter      categories      region      isBrandList      filterBooking      hasNearQuery      isPublicMask      __typename    }    __typename  }}"}]'
    r = requests.post(url, data=payload.encode("utf-8"), headers=header, cookies=cookie)
    r = json.loads(r.text)
    items = r[0]["data"]["rests"]["businesses"]["items"]
    for item in items:
        exist = False
        rest_str = ""
        this_hosp_name = item["name"]
        for vaccine in item["vaccineQuantity"]["list"]:
            if vaccine["quantity"] > 0 and (vaccine["vaccineType"]=="화이자" or vaccine["vaccineType"]=="모더나"):
                exist = True
                rest_str += " ⌙ {} : {}개 남음".format(vaccine["vaccineType"], vaccine["quantity"])
        if exist:
            done = True
            playsound("wow.wav")
            print(this_hosp_name)
            print("{}\n".format(rest_str))

In [ ]:
done = False
try_count = 0
cookie = load_cookie_from_chrome()
while True:
    if done:
        break
    try_count += 1
    time.sleep(0.2)
    print("{}번째 시도중...".format(try_count), end="\r")
    find_vaccine(cookie)